In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic

In [ ]:
#!pip install geopy

In [ ]:
df = pd.read_csv('base_suja/base_unificada_suja.csv')

In [ ]:
df.columns

In [ ]:
colunas = [
    'data_evento'
    , 'ano_evento'
    , 'evento_MUNNOMEX'
    , 'evento_SIGLA_UF'
    , 'sum_CENTROBS'
    , 'sum_QTINST34'
    , 'sum_QTINST35'
    , 'sum_QTINST36'
    , 'sum_QTINST37'
    , 'sum_QTLEIT34'
    , 'sum_QTLEIT38'
    , 'sum_QTLEIT39'
    , 'sum_QTLEIT40'
    , 'sum_CENTRNEO'
    , 'TP_UNID_5'
    , 'TP_UNID_7'
    , 'TP_UNID_15'
    , 'TP_UNID_36'
    , 'FLAG_BASE'
]
df = df[colunas]

In [ ]:
df['mes_ano_evento'] = [str(i)[0:7] for i in df['data_evento']]

In [ ]:
df_agrupado = df.groupby(
 [
    'evento_SIGLA_UF'
    , 'evento_MUNNOMEX'
    , 'ano_evento'
 ]
    , as_index=False
).agg(
    QTINST34=pd.NamedAgg(column='sum_QTINST34', aggfunc='median')
    , QTINST35=pd.NamedAgg(column='sum_QTINST35', aggfunc='median')
    , QTINST36=pd.NamedAgg(column='sum_QTINST36', aggfunc='median')
    , QTINST37=pd.NamedAgg(column='sum_QTINST37', aggfunc='median')
    , QTLEIT34=pd.NamedAgg(column='sum_QTLEIT34', aggfunc='median')
    , QTLEIT38=pd.NamedAgg(column='sum_QTLEIT38', aggfunc='median')
    , QTLEIT39=pd.NamedAgg(column='sum_QTLEIT39', aggfunc='median')
    , QTLEIT40=pd.NamedAgg(column='sum_QTLEIT40', aggfunc='median')
    , TP_UNID_5=pd.NamedAgg(column='TP_UNID_5', aggfunc='median') 
    , TP_UNID_7=pd.NamedAgg(column='TP_UNID_7', aggfunc='median') 
    , TP_UNID_15=pd.NamedAgg(column='TP_UNID_15', aggfunc='median') 
    , TP_UNID_36=pd.NamedAgg(column='TP_UNID_36', aggfunc='median') 
    , QTD_NASCIMENTOS=pd.NamedAgg(column='data_evento', aggfunc='size') 
    )

In [ ]:
df_agrupado.to_csv('check_leitos_vs_nascimentos.csv', index=False)

In [2]:
df_agrupado = pd.read_csv('check_leitos_vs_nascimentos.csv')

In [3]:
df_agrupado['QTINST'] = df_agrupado['QTINST34'] + df_agrupado['QTINST35'] + df_agrupado['QTINST36'] + df_agrupado['QTINST37']
df_agrupado['QTLEIT'] = df_agrupado['QTLEIT34'] + df_agrupado['QTLEIT38'] + df_agrupado['QTLEIT39'] + df_agrupado['QTLEIT40']

In [4]:
# Variação
df_agrupado['var_QTD_NASCIMENTOS'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['QTD_NASCIMENTOS'].pct_change()

df_agrupado['var_QTINST'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['QTINST'].pct_change()

df_agrupado['var_QTLEIT'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['QTLEIT'].pct_change()

df_agrupado['var_TP_UNID_5'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['TP_UNID_5'].pct_change()

df_agrupado['var_TP_UNID_7'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['TP_UNID_7'].pct_change()

df_agrupado['var_TP_UNID_15'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['TP_UNID_15'].pct_change()

df_agrupado['var_TP_UNID_36'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['TP_UNID_36'].pct_change()

In [5]:
# Diferença
df_agrupado['dif_QTD_NASCIMENTOS'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['QTD_NASCIMENTOS'].diff()

df_agrupado['dif_QTINST'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['QTINST'].diff()

df_agrupado['dif_QTLEIT'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['QTLEIT'].diff()

df_agrupado['dif_TP_UNID_5'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['TP_UNID_5'].diff()

df_agrupado['dif_TP_UNID_7'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['TP_UNID_7'].diff()

df_agrupado['dif_TP_UNID_15'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['TP_UNID_15'].diff()

df_agrupado['dif_TP_UNID_36'] = df_agrupado.sort_values(
  by=['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento']
).groupby(
  ['evento_SIGLA_UF', 'evento_MUNNOMEX']
)['TP_UNID_36'].diff()

In [9]:
# Removendo os missings
df_agrupado = df_agrupado.fillna(0)

In [10]:
# Separando municípios que tiveram menos que 500 nascimentos em um ano
df_agrupado['chave_mun_uf_evento'] = [f'{i}_{j}' for i, j in zip(df_agrupado['evento_SIGLA_UF'], df_agrupado['evento_MUNNOMEX'])]
df_nasc_menor_500 = df_agrupado[df_agrupado['QTD_NASCIMENTOS'] <= 500]
lista_mun_interesse = pd.unique(df_nasc_menor_500['chave_mun_uf_evento'])

In [11]:
# 4004 de um total de 4761
df_interesse = df_agrupado[df_agrupado['chave_mun_uf_evento'].isin(lista_mun_interesse)]
df_interesse = df_interesse.reset_index(drop=True)

In [12]:
colunas = [
    'evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento', 'QTINST', 'QTLEIT', 'TP_UNID_5', 'TP_UNID_7'
    , 'TP_UNID_15', 'TP_UNID_36', 'QTD_NASCIMENTOS', 'var_QTINST', 'var_QTLEIT', 'var_TP_UNID_5'
    , 'var_TP_UNID_7', 'var_TP_UNID_15', 'var_TP_UNID_36', 'var_QTD_NASCIMENTOS'
]
# Removendo os missings
df_interesse = df_interesse[colunas]

In [13]:
# Lista para armazenar as correlações todos os anos
correlacoes = []
ufs =  pd.unique(df_interesse['evento_SIGLA_UF'])
# Agrupar os dados por município e calcular a correlação
for uf in ufs:
    subset = df_interesse.loc[(df_interesse['evento_SIGLA_UF'] == uf) & (df_interesse['ano_evento'] != 2018)]
    if len(subset) > 1:  # Certifique-se de que há dados suficientes para calcular a correlação
        correlacao_QTLEIT = subset[['var_QTLEIT', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_QTINST = subset[['var_QTINST', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_TP_UNID_5 = subset[['var_TP_UNID_5', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_TP_UNID_7 = subset[['var_TP_UNID_7', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_TP_UNID_15 = subset[['var_TP_UNID_15', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_TP_UNID_36 = subset[['var_TP_UNID_36', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacoes.append({'evento_SIGLA_UF': uf, 'correlacao_QTLEIT': correlacao_QTLEIT
                           , 'correlacao_QTINST': correlacao_QTINST, 'correlacao_TP_UNID_5': correlacao_TP_UNID_5
                           , 'correlacao_TP_UNID_7': correlacao_TP_UNID_7, 'correlacao_TP_UNID_15': correlacao_TP_UNID_15
                           , 'correlacao_TP_UNID_36': correlacao_TP_UNID_36})

# Criar um DataFrame com os resultados
df_correlacoes = pd.DataFrame(correlacoes)
aa=df_correlacoes[(df_correlacoes['correlacao_QTLEIT'] >= 0.5) | (df_correlacoes['correlacao_QTINST'] >= 0.5) |
              (df_correlacoes['correlacao_TP_UNID_5'] >= 0.5) | (df_correlacoes['correlacao_TP_UNID_7'] >= 0.5) |
              (df_correlacoes['correlacao_TP_UNID_15'] >= 0.5) | (df_correlacoes['correlacao_TP_UNID_36'] >= 0.5)]

In [14]:
# Considerando variações positivas e negativas tenho somente uma uf que se observa euma forte correlação
aa

,evento_SIGLA_UF,correlacao_QTLEIT,correlacao_QTINST,correlacao_TP_UNID_5,correlacao_TP_UNID_7,correlacao_TP_UNID_15,correlacao_TP_UNID_36
22,SC,0.941766,0.007365,0.006919,NaN,0.003804,-0.011198


In [15]:
# grafico
# Criar gráficos de dispersão 
#for uf in aa['evento_SIGLA_UF']:
#    subset = df_interesse[df_interesse['evento_SIGLA_UF'] == uf]
#    corr = round(df_correlacoes.loc[df_correlacoes['evento_SIGLA_UF'] == uf, 'correlacao_QTLEIT'], 2)
#    plt.figure(figsize=(10, 6))
#    plt.scatter(subset['var_QTLEIT'], subset['var_QTD_NASCIMENTOS'], alpha=0.5)
#    plt.title(f'Leitos VS Nasc corr {corr.reset_index(drop=True)[0]} municipio {uf} ')
#    plt.xlabel('Número de Leitos (var_QTDLEIT)')
#    plt.ylabel('Número de Nascimentos (var_qtd_nasc)')
#    plt.grid(True)
#    plt.show()

In [16]:
# Lista para armazenar as correlações todos os anos
correlacoes_ano = []
ufs =  pd.unique(df_interesse['evento_SIGLA_UF'])
# Agrupar os dados por município e calcular a correlação
for uf in ufs:
    for ano in [2019, 2020, 2021, 2022]:
        subset = df_interesse.loc[(df_interesse['evento_SIGLA_UF'] == uf) & (df_interesse['ano_evento'] == ano)]
        if len(subset) > 1:  # Certifique-se de que há dados suficientes para calcular a correlação
            correlacao_QTLEIT = subset[['var_QTLEIT', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_QTINST = subset[['var_QTINST', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_TP_UNID_5 = subset[['var_TP_UNID_5', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_TP_UNID_7 = subset[['var_TP_UNID_7', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_TP_UNID_15 = subset[['var_TP_UNID_15', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_TP_UNID_36 = subset[['var_TP_UNID_36', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacoes_ano.append({'evento_SIGLA_UF': uf, 'ano_evento': ano,'correlacao_QTLEIT': correlacao_QTLEIT
                               , 'correlacao_QTINST': correlacao_QTINST, 'correlacao_TP_UNID_5': correlacao_TP_UNID_5
                               , 'correlacao_TP_UNID_7': correlacao_TP_UNID_7, 'correlacao_TP_UNID_15': correlacao_TP_UNID_15
                               , 'correlacao_TP_UNID_36': correlacao_TP_UNID_36})

# Criar um DataFrame com os resultados
df_correlacoes_ano = pd.DataFrame(correlacoes_ano)
bb=df_correlacoes_ano[(df_correlacoes_ano['correlacao_QTLEIT'] >= 0.5) | (df_correlacoes_ano['correlacao_QTINST'] >= 0.5) |
              (df_correlacoes_ano['correlacao_TP_UNID_5'] >= 0.5) | (df_correlacoes_ano['correlacao_TP_UNID_7'] >= 0.5) |
              (df_correlacoes_ano['correlacao_TP_UNID_15'] >= 0.5) | (df_correlacoes_ano['correlacao_TP_UNID_36'] >= 0.5)]

In [17]:
bb

,evento_SIGLA_UF,ano_evento,correlacao_QTLEIT,correlacao_QTINST,correlacao_TP_UNID_5,correlacao_TP_UNID_7,correlacao_TP_UNID_15,correlacao_TP_UNID_36
19,BA,2022,0.713484,0.591069,-0.019084,0.010401,0.014231,0.012766
69,RJ,2020,-0.274086,-0.111427,-0.030857,0.997754,NaN,-0.217234
79,RO,2022,0.029778,NaN,0.609729,NaN,-0.220128,-0.023651
91,SC,2022,0.989492,0.018186,0.018768,NaN,0.005237,-0.022781


In [18]:
# agora vou considerar somente as variações negativas
# Lista para armazenar as correlações todos os anos
correlacoes = []
ufs =  pd.unique(df_interesse['evento_SIGLA_UF'])
# Agrupar os dados por município e calcular a correlação
for uf in ufs:
    subset = df_interesse.loc[(df_interesse['evento_SIGLA_UF'] == uf) & (df_interesse['ano_evento'] != 2018) &
                              (df_interesse['var_QTD_NASCIMENTOS'] < 0)]
    if len(subset) > 1:  # Certifique-se de que há dados suficientes para calcular a correlação
        correlacao_QTLEIT = subset[['var_QTLEIT', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_QTINST = subset[['var_QTINST', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_TP_UNID_5 = subset[['var_TP_UNID_5', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_TP_UNID_7 = subset[['var_TP_UNID_7', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_TP_UNID_15 = subset[['var_TP_UNID_15', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacao_TP_UNID_36 = subset[['var_TP_UNID_36', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
        correlacoes.append({'evento_SIGLA_UF': uf, 'correlacao_QTLEIT': correlacao_QTLEIT
                           , 'correlacao_QTINST': correlacao_QTINST, 'correlacao_TP_UNID_5': correlacao_TP_UNID_5
                           , 'correlacao_TP_UNID_7': correlacao_TP_UNID_7, 'correlacao_TP_UNID_15': correlacao_TP_UNID_15
                           , 'correlacao_TP_UNID_36': correlacao_TP_UNID_36})

# Criar um DataFrame com os resultados
df_correlacoes = pd.DataFrame(correlacoes)
aa_=df_correlacoes[(df_correlacoes['correlacao_QTLEIT'] >= 0.5) | (df_correlacoes['correlacao_QTINST'] >= 0.5) |
              (df_correlacoes['correlacao_TP_UNID_5'] >= 0.5) | (df_correlacoes['correlacao_TP_UNID_7'] >= 0.5) |
              (df_correlacoes['correlacao_TP_UNID_15'] >= 0.5) | (df_correlacoes['correlacao_TP_UNID_36'] >= 0.5)]

In [19]:
aa_

,evento_SIGLA_UF,correlacao_QTLEIT,correlacao_QTINST,correlacao_TP_UNID_5,correlacao_TP_UNID_7,correlacao_TP_UNID_15,correlacao_TP_UNID_36


In [20]:
# Lista para armazenar as correlações todos os anos e somente variacoes negativas
correlacoes_ano = []
ufs =  pd.unique(df_interesse['evento_SIGLA_UF'])
# Agrupar os dados por município e calcular a correlação
for uf in ufs:
    for ano in [2019, 2020, 2021, 2022]:
        subset = df_interesse.loc[(df_interesse['evento_SIGLA_UF'] == uf) & (df_interesse['ano_evento'] == ano) &
                                 (df_interesse['var_QTD_NASCIMENTOS'] < 0)]
        if len(subset) > 1:  # Certifique-se de que há dados suficientes para calcular a correlação
            correlacao_QTLEIT = subset[['var_QTLEIT', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_QTINST = subset[['var_QTINST', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_TP_UNID_5 = subset[['var_TP_UNID_5', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_TP_UNID_7 = subset[['var_TP_UNID_7', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_TP_UNID_15 = subset[['var_TP_UNID_15', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacao_TP_UNID_36 = subset[['var_TP_UNID_36', 'var_QTD_NASCIMENTOS']].corr().iloc[0, 1]
            correlacoes_ano.append({'evento_SIGLA_UF': uf, 'ano_evento': ano,'correlacao_QTLEIT': correlacao_QTLEIT
                               , 'correlacao_QTINST': correlacao_QTINST, 'correlacao_TP_UNID_5': correlacao_TP_UNID_5
                               , 'correlacao_TP_UNID_7': correlacao_TP_UNID_7, 'correlacao_TP_UNID_15': correlacao_TP_UNID_15
                               , 'correlacao_TP_UNID_36': correlacao_TP_UNID_36})

# Criar um DataFrame com os resultados
df_correlacoes_ano = pd.DataFrame(correlacoes_ano)
bb_=df_correlacoes_ano[(df_correlacoes_ano['correlacao_QTLEIT'] >= 0.5) | (df_correlacoes_ano['correlacao_QTINST'] >= 0.5) |
              (df_correlacoes_ano['correlacao_TP_UNID_5'] >= 0.5) | (df_correlacoes_ano['correlacao_TP_UNID_7'] >= 0.5) |
              (df_correlacoes_ano['correlacao_TP_UNID_15'] >= 0.5) | (df_correlacoes_ano['correlacao_TP_UNID_36'] >= 0.5)]

In [21]:
bb_

,evento_SIGLA_UF,ano_evento,correlacao_QTLEIT,correlacao_QTINST,correlacao_TP_UNID_5,correlacao_TP_UNID_7,correlacao_TP_UNID_15,correlacao_TP_UNID_36
14,AP,2021,NaN,NaN,NaN,NaN,NaN,0.667254
46,MT,2021,0.467139,0.417937,0.511460,NaN,NaN,0.152306
77,RO,2020,NaN,-0.279705,-0.285785,NaN,-0.049962,0.534710
78,RO,2021,NaN,NaN,NaN,NaN,NaN,0.554326


# Variação entre municípios

In [49]:
# Calcular a distância entre municipios selecionar os mais próximos dos municípios que tiveram queda e
# verificar se tiveram aumento no número de nascimentos.
base_lat_long_mun = pd.read_csv('base_limpa/latitude_longitude_municipios.csv', sep=';', decimal=',')

In [50]:
base_lat_long_mun['mun_uf'] = [f'{i}_{j}' for i, j in zip(base_lat_long_mun['mun_MUNNOMEX'], base_lat_long_mun['uf_SIGLA_UF'])]

In [ ]:
# Obtendo as UFs únicas
#ufs = pd.unique(base_lat_long_mun['uf_SIGLA_UF'])
lista = []

# Iterando por cada UF
#for uf in ufs:
#df_uf = base_lat_long_mun[base_lat_long_mun['uf_SIGLA_UF'] == uf]
munic = pd.unique(base_lat_long_mun['mun_uf']).tolist()
pares_processados = set()  # Usado para armazenar pares já processados
# Iterando por cada município
for i, mun in enumerate(munic):
    for mun_2 in munic[i+1:]:
        # Verificando se o par já foi processado
        par = tuple(sorted([mun, mun_2]))
        if par in pares_processados:
            continue

        # Coordenadas do primeiro município
        lat1 = base_lat_long_mun.loc[base_lat_long_mun['mun_uf'] == mun, 'mun_LATITUDE'].values[0]
        long1 = base_lat_long_mun.loc[base_lat_long_mun['mun_uf'] == mun, 'mun_LONGITUDE'].values[0]

        # Coordenadas do segundo município
        lat2 = base_lat_long_mun.loc[base_lat_long_mun['mun_uf'] == mun_2, 'mun_LATITUDE'].values[0]
        long2 = base_lat_long_mun.loc[base_lat_long_mun['mun_uf'] == mun_2, 'mun_LONGITUDE'].values[0]

        # Calculando a distância
        distancia = geodesic((lat1, long1), (lat2, long2)).kilometers

        # Criando o dicionário com os dados do par
        dicionario = {
            'municipio1':[mun], 'municipio2': [mun_2], 'dist_km': [distancia],
            'lat_mun1': [lat1], 'long_mun1': [long1],
            'lat_mun2': [lat2], 'long_mun2': [long2]
        }
        lista.append(dicionario)

        # Adicionando o par ao conjunto de pares processados
        pares_processados.add(par)

In [ ]:
lista = [pd.DataFrame(i) for i in lista]
lista = pd.concat(lista)

In [ ]:
len(lista)

In [ ]:
lista.to_csv('distancia_municipios_uf_v2.csv', sep=';', decimal=',')

In [37]:
lista = pd.read_csv('base_limpa/distancia_municipios.csv', sep=';', decimal=',')
munic1 = lista['municipio2']
munic2 = lista['municipio1']
lista2 = lista
lista2['municipio1']=munic2
lista2['municipio2']=munic1
lista = pd.concat([lista, lista2]).reset_index(drop=True)
# selecionando somente municípios com até 100 km de distância
lista_100 = lista[lista['dist_km'] <= 100.99].sort_values(['municipio1', 'dist_km']).reset_index(drop=True)

In [33]:
variacoes_negativas = df_agrupado.loc[(df_agrupado['ano_evento'] != 2018) &
                              (df_agrupado['var_QTD_NASCIMENTOS'] < 0)].reset_index(drop=True)

In [34]:
#variacoes_positivas = df_agrupado.loc[(df_agrupado['ano_evento'] != 2018) &
#                                      (df_agrupado['var_QTD_NASCIMENTOS'] > 0)].reset_index(drop=True)

In [40]:
# Vou pegar a chave
chave_uf_mun = pd.unique(variacoes_negativas['chave_mun_uf_evento'])
colunas = [
    'chave_mun_uf_evento'
    , 'ano_evento'
    , 'QTD_NASCIMENTOS'
    , 'var_QTD_NASCIMENTOS'
    , 'dif_QTD_NASCIMENTOS'
]
for munic in chave_uf_mun:
    muncipio_transf = lista_100.loc[lista_100['municipio1']==munic, 'municipio2']
    df_perde = df_agrupado.loc[(df_agrupado['ano_evento'] != 2018) &
                             (df_agrupado'chave_mun_uf_evento'] == munic), colunas].reset_index(drop=True)
    df_ganha = df_agrupado.loc[(df_agrupado['ano_evento'] != 2018) &
                             (df_agrupado'chave_mun_uf_evento'].isin(muncipio_transf)), colunas].reset_index(drop=True)
    df_ganha.columns = [f'compara_{i}' for i in df_ganha.columns]
    df_perde = df_perde.merge(df_ganha, how='left', left_on='ano_evento', right_on='compara_ano_evento')
    
    
    

In [ ]:
# Calcular os deltas e depois fazer a correlação entre os municipios para variação e diferença

In [35]:
variacoes_positivas.columns

Index(['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento', 'QTINST34',
       'QTINST35', 'QTINST36', 'QTINST37', 'QTLEIT34', 'QTLEIT38', 'QTLEIT39',
       'QTLEIT40', 'TP_UNID_5', 'TP_UNID_7', 'TP_UNID_15', 'TP_UNID_36',
       'QTD_NASCIMENTOS', 'QTINST', 'QTLEIT', 'var_QTD_NASCIMENTOS',
       'var_QTINST', 'var_QTLEIT', 'var_TP_UNID_5', 'var_TP_UNID_7',
       'var_TP_UNID_15', 'var_TP_UNID_36', 'dif_QTD_NASCIMENTOS', 'dif_QTINST',
       'dif_QTLEIT', 'dif_TP_UNID_5', 'dif_TP_UNID_7', 'dif_TP_UNID_15',
       'dif_TP_UNID_36', 'chave_mun_uf_evento'],
      dtype='object')

In [36]:
variacoes_negativas.columns

Index(['evento_SIGLA_UF', 'evento_MUNNOMEX', 'ano_evento', 'QTINST34',
       'QTINST35', 'QTINST36', 'QTINST37', 'QTLEIT34', 'QTLEIT38', 'QTLEIT39',
       'QTLEIT40', 'TP_UNID_5', 'TP_UNID_7', 'TP_UNID_15', 'TP_UNID_36',
       'QTD_NASCIMENTOS', 'QTINST', 'QTLEIT', 'var_QTD_NASCIMENTOS',
       'var_QTINST', 'var_QTLEIT', 'var_TP_UNID_5', 'var_TP_UNID_7',
       'var_TP_UNID_15', 'var_TP_UNID_36', 'dif_QTD_NASCIMENTOS', 'dif_QTINST',
       'dif_QTLEIT', 'dif_TP_UNID_5', 'dif_TP_UNID_7', 'dif_TP_UNID_15',
       'dif_TP_UNID_36', 'chave_mun_uf_evento'],
      dtype='object')

In [39]:
variacoes_negativas

,evento_SIGLA_UF,evento_MUNNOMEX,ano_evento,QTINST34,QTINST35,QTINST36,QTINST37,QTLEIT34,QTLEIT38,QTLEIT39,...,var_TP_UNID_15,var_TP_UNID_36,dif_QTD_NASCIMENTOS,dif_QTINST,dif_QTLEIT,dif_TP_UNID_5,dif_TP_UNID_7,dif_TP_UNID_15,dif_TP_UNID_36,chave_mun_uf_evento
0,AC,ACRELANDIA,2019,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0,AC_ACRELANDIA
1,AC,ACRELANDIA,2022,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0,AC_ACRELANDIA
2,AC,ASSIS BRASIL,2020,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-45.0,0.0,0.0,0.0,0.0,0.0,0.0,AC_ASSIS BRASIL
3,AC,ASSIS BRASIL,2022,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-39.0,0.0,0.0,0.0,0.0,0.0,0.0,AC_ASSIS BRASIL
4,AC,BRASILEIA,2020,0.0,5.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,-42.0,5.0,-1.0,0.0,0.0,0.0,0.0,AC_BRASILEIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7844,TO,WANDERLANDIA,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,TO_WANDERLANDIA
7845,TO,WANDERLANDIA,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,TO_WANDERLANDIA
7846,TO,XAMBIOA,2019,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,-40.0,0.0,0.0,0.0,0.0,0.0,0.0,TO_XAMBIOA
7847,TO,XAMBIOA,2020,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,-21.0,0.0,0.0,0.0,0.0,0.0,0.0,TO_XAMBIOA


In [44]:
lista_100

,Unnamed: 0,municipio1,municipio2,dist_km,uf,lat_mun1,long_mun1,lat_mun2,long_mun2
91438,0,SANTA MARIA DA VITORIA,SAO FELIX DO CORIBE,0.549012,BA,-13.398,-44.198,-13.402,-44.195
19417,0,AURELINO LEAL,UBAITABA,0.618138,BA,-14.317,-39.327,-14.313,-39.323
858799,0,CAPINZAL,OURO,0.680444,SC,-27.344,-51.612,-27.341,-51.618
152421,0,CACHOEIRA GRANDE,PRESIDENTE JUSCELINO,0.786043,MA,-2.927,-44.058,-2.928,-44.065
427301,0,JANAUBA,NOVA PORTEIRINHA,0.857109,MG,-15.803,-43.309,-15.803,-43.301
...,...,...,...,...,...,...,...,...,...
986950,0,EUCLIDES DA CUNHA PAULISTA,PRESIDENTE EPITACIO,100.987655,SP,-22.561,-52.590,-21.763,-52.116
556666,0,ARACAGI,NOVA FLORESTA,100.988393,PB,-6.853,-35.381,-6.455,-36.203
3491,0,JEQUIA DA PRAIA,JOAQUIM GOMES,100.989350,AL,-10.004,-36.024,-9.133,-35.748
25500,0,BOA VISTA DO TUPIM,BREJOES,100.989596,BA,-12.660,-40.609,-13.104,-39.796
